In [114]:
#Importing Libraries
import csv
#import paramaters
from parsel import Selector
from time import sleep
from time import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import numpy as np
from numpy.random import randint
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from itertools import repeat

In [171]:
terms = [('Mining',9), ('Infrastructure',7), ('Engineer',6), ('PMP',5)]

range_ = range(1,len(terms))
{term: rank for term, rank in zip(terms,range(1,len(terms)+1))}


{'Engineer': 3, 'Infrastructure': 2, 'Mining': 1, 'PMP': 4, 'alala': 5}

In [174]:
" And ".join(terms)

'Mining And Infrastructure And Engineer And PMP And alala'

In [211]:
class googlelink_ranks():
    def __init__(self,position,location,firms,secondary_terms):
        self.position = position
        self.location = location
        self.firms = firms
        self.secondary_terms = secondary_terms
        self.search_criteria_dict = { "positions" : self.position , "location" : self.location ,
                                     "firms" : self.firms , "secondary_terms" : self.secondary_terms.keys() }
    
    def custom_sleep(self, min_=4,max_=9):
        sleep(randint(min_,max_)+np.random.rand())
        
    def construct_search(self):
        # ADD QUOTES
        add_quotes = lambda term: '"'+term+'"'
        for key, var in self.search_criteria_dict.items():
            self.search_criteria_dict[key] = list(map(add_quotes, var))

        # ADD SEARCH TERMS TO BASE
        for key, var in self.search_criteria_dict.items():
            if key == 'secondary_terms':
                continue
            self.search_criteria_dict[key] = ' OR '.join(var)
        
        # Base term + secondary terms
        search_dict = {}
        base_term = 'site:linkedin.com/in/'
        for term in self.search_criteria_dict['secondary_terms']:
            extended_term = base_term + " " + self.search_criteria_dict["positions"] + " AND " + term + " AND " + self.search_criteria_dict['firms'] + " AND " + self.search_criteria_dict['location']   
            search_dict[term.strip('"')] = extended_term  #search_dict is a dictionary with key: secondary terms and value is the link 
        self.search_dict = search_dict
        
    def get_profile_links(self,query, google_pages=10, headless=True, path='chromedriver'):
        ############ INITIALIZE RANDOM SEED ############
        np.random.seed(randint(1000))
        ############ INITIALIZE DRIVER ############
        options = webdriver.ChromeOptions()
        if headless:
            options.add_argument('headless')
        #set drive
        driver = webdriver.Chrome(executable_path=path, chrome_options=options)
        t0_full = time()

        ############ RUN GOOGLE QUERY ############
        print("Running google query: {} ".format(query))
        driver.get('https://www.google.com')
        self.custom_sleep()
        search_query = driver.find_element_by_name('q')
        search_query.send_keys(query)
        self.custom_sleep()
        search_query.send_keys(Keys.RETURN)

        ############ SCRAPE GOOGLE LINKS ############
        linkedin_urls = []
        for i in range(google_pages):
            t0 = time()
            linkedin_urls_single = driver.find_elements_by_tag_name('cite')
            linkedin_urls_single = [(url.text,i+1) for url in linkedin_urls_single]
            linkedin_urls += linkedin_urls_single
            self.custom_sleep()
            print("Scrape page {} complete, runtime = {}".format(i+1,time()-t0), end="\r")
            max_page= i+1
            try:
                next_page = driver.find_element_by_xpath('//*[@id="pnnext"]')
                next_page.click()
            except:
                break
            
        print("{0} google pages scraped. Full script runtime = {1}".format(max_page,time() - t0_full))
        return linkedin_urls

    def get_links_dict(self):
        # Extract google search link for every secondary search term
        final_dict = {}
        for key in self.search_dict:
            final_dict[key] = self.get_profile_links(query = self.search_dict[key]) #final_dict's keys correspond to secondary terms and values correspond to links wihtin its secondary tern search respective
        return final_dict
        
    def get_unique_links(self, tuple_dict):
        # Extract list of unique links
        unique_links = set()
        for key,val in tuple_dict.items():
            unique_links.update([tup[0] for tup in val]) #converting a dictionary of links to a list of unique links
        return unique_links
    
    def extract_score(self, unique_links, final_dict, alpha=0.5):
        # Extract scores and links
        link_list = []
        for link in unique_links: 
            final_score = 0 #scoring links based on the number of search 'pool' they fall into
            category = [] #labeling links on which search category they appear in
            for key, tuples in final_dict.items():
                only_links = [tup[0] for tup in tuples]
                if link in only_links:
                    # Extract second element of corresponding link (page occured in google) 
                    page_occured = tuples[only_links.index(link)][1]
                    term_rank = self.secondary_terms[key]
                    term_score = ((1/page_occured)*alpha) + ((term_rank/10)*(1-alpha))
                    final_score+= term_score
                    category.append((key, page_occured, term_rank, term_score))
            #converting each link with their scores and category to a tuple
            link_list.append((link,final_score,category)) # each tuple should have format (str, int, list)
        return link_list
            
    def construct_final_list(self, existing_data=None): 
        if existing_data==None:
            self.final_dict = self.get_links_dict()
        else:
            # LOAD IN EXISTING DATA
            ################# WRITE CODE HERE #####################
        # Extract unique links
        unique_links = self.get_unique_links(self.final_dict) 
        # Extract scores
        link_tuples = self.extract_score(unique_links, self.final_dict)
        return link_tuples
        
    def write_csv(self):
        with open('ScrapingResults.csv', 'wb') as f:  # Just use 'w' mode in 3.x
            w = csv.DictWriter(f, self.final_dict.keys())
            w.writeheader()
            w.writerow(self.final_dict)

IndentationError: expected an indented block (<ipython-input-211-1eba69d8d8ab>, line 112)

In [207]:
tuples[0]

('https://www.linkedin.com/in/kristen-palmer-cpa-ca-pmp-7921811a',
 0.41666666666666663,
 [('PMP', 3, 5)])

In [204]:
positions = ['Senior manager', 'Director']
locations = ['Toronto']
firm = ['PwC', 'Deloitte', 'EY']

secondary_terms = {'Mining':9,
                   'Infrastructure':7, 
                   'Engineer':6,
                   'PMP':5}

scrape_links = googlelink_ranks(positions,locations,firm,secondary_terms)
scrape_links.construct_search()
tuples = scrape_links.construct_final_list()

Running google query: site:linkedin.com/in/ "Senior manager" OR "Director" AND "Mining" AND "PwC" OR "Deloitte" OR "EY" AND "Toronto" 
10 google pages scraped. Full script runtime = 92.73594355583191
Running google query: site:linkedin.com/in/ "Senior manager" OR "Director" AND "Infrastructure" AND "PwC" OR "Deloitte" OR "EY" AND "Toronto" 
10 google pages scraped. Full script runtime = 84.68955659866333
Running google query: site:linkedin.com/in/ "Senior manager" OR "Director" AND "Engineer" AND "PwC" OR "Deloitte" OR "EY" AND "Toronto" 
10 google pages scraped. Full script runtime = 92.92075896263123
Running google query: site:linkedin.com/in/ "Senior manager" OR "Director" AND "PMP" AND "PwC" OR "Deloitte" OR "EY" AND "Toronto" 
10 google pages scraped. Full script runtime = 96.580326795578


In [213]:
df = pd.DataFrame(tuples, columns=['link','score','terms'])
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 1000)
df2 = df.sort_values('score',ascending=False).reset_index()

In [217]:
writer = pd.ExcelWriter('Exec_search_mining.xlsx')
df2.to_excel(writer, 'sheet1')
writer.save()

In [93]:
positions = ['Senior manager', 'Director']
locations = ['Toronto']
firm = ['PwC', 'Deloitte', 'EY']
secondary_terms = ['Mining', 'Engineer' , 'PMP']

scrape_links = googlelink_ranks(positions,locations,firm,secondary_terms)
scrape_links.construct_links()
scrape_links.construct_final_list()
#scrape_links.link_list
#scrape_links.write_csv

"Senior manager" OR "Director"
"Toronto"
"PwC" OR "Deloitte" OR "EY"
"Mining"
"Engineer"
"PMP"
Running google query: site:linkedin.com/in/ "Senior manager" OR "Director" AND "Mining" AND "PwC" OR "Deloitte" OR "EY" AND "Toronto" 
10 google pages scraped. Full script runtime = 86.38096499443054
Running google query: site:linkedin.com/in/ "Senior manager" OR "Director" AND "Engineer" AND "PwC" OR "Deloitte" OR "EY" AND "Toronto" 
10 google pages scraped. Full script runtime = 96.21447777748108
Running google query: site:linkedin.com/in/ "Senior manager" OR "Director" AND "PMP" AND "PwC" OR "Deloitte" OR "EY" AND "Toronto" 
10 google pages scraped. Full script runtime = 89.18608021736145


In [101]:
df = pd.DataFrame(scrape_links.link_list, columns=['link','score','terms'])
df.head()

,link,score,terms
0,https://ca.linkedin.com/in/mary-sanagan-664b6017,1,"[""PMP""]"
1,https://ca.linkedin.com/in/jlising,1,"[""Mining""]"
2,https://ca.linkedin.com/in/macmcdonald1,1,"[""Mining""]"
3,https://ca.linkedin.com/in/mikesamis-scmdecisi...,1,"[""Mining""]"
4,https://www.linkedin.com/in/kristen-palmer-cpa...,1,"[""PMP""]"
